# OSTk Cross-Platform Validation Against GMAT/Orekit (Mission Sequence Scenarios 1)

This tutorial demonstrates how to compare OSTk numerical orbit propagation with other tools like Orekit or GMAT. This example will be for a mission sequence that consists of a coasting phases.

In [35]:
import numpy as np
import pandas as pd
import csv
import os
import plotly.graph_objs as go
from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.display import create_2d_map, create_3d_globe
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.trajectory import LocalOrbitalFrameFactory
from ostk.astrodynamics.trajectory import LocalOrbitalFrameDirection
from ostk.astrodynamics.trajectory.state import CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianVelocity
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.guidance_law import ConstantThrust
from ostk.astrodynamics.trajectory import Orbit, Propagator, State
from ostk.astrodynamics.trajectory.orbit.models import Propagated
from ostk.core.filesystem import Directory
from ostk.mathematics.geometry.d3.objects import Composite, Cuboid, Point
from ostk.physics import Environment
from ostk.physics.coordinate import Frame, Position, Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Interval, Scale
from ostk.physics.units import Mass

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point


## Set up Comparison Files

Define array inputs that can be changed to include/exclude gmat/orekit result comparisons

In [36]:
filenames = ["gmat_astrodynamics/scenario001-mission-sequence.csv", "orekit_astrodynamics/scenario001-mission-sequence.csv"]
legend_name_list = ["GMAT","OREKIT"]

multiplication_factors = [np.array([1.0e3, 1.0e3, 1.0e3]), np.array([1.0, 1.0, 1.0])]


## Setup Comparison Scenario in OSTk

Define the physical `Environment`

In [37]:
earth = Earth.from_models(
    EarthGravitationalModel(
        EarthGravitationalModel.Type.EGM96, Directory.undefined(), 0, 0
    ),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Undefined),
)
environment = Environment(Instant.J2000(), [earth])


Define the `SatelliteSystem` 

In [38]:
mass = Mass(100.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(
    Cuboid(
        Point(0.0, 0.0, 0.0),
        np.eye(3).tolist(),
        [1.0, 0.0, 0.0],
    )
)
inertia_tensor = np.identity(3)
surface_area = 1.0
drag_coefficient = 2.2

satellite_system = SatelliteSystem(
    mass,
    satellite_geometry,
    inertia_tensor,
    surface_area,
    drag_coefficient,
)


Define the initial `State` of the `SatelliteSystem`

In [39]:
start_instant = Instant.date_time(DateTime.parse("2023-01-01T00:00:00.000"), Scale.UTC)
initial_state = State(
    start_instant,
    Position.meters(
        np.array([-4283387.412456234, -4451426.776125099, -2967617.850750064]),
        Frame.GCRF(),
    ),
    Velocity.meters_per_second(
        np.array([4948.074939732172, -957.3429532772123, -5721.173027553033]),
        Frame.GCRF(),
    ),
)


In [40]:
state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    mass.in_kilograms(),
    satellite_system.get_cross_sectional_surface_area(),
    satellite_system.get_drag_coefficient(),
]
start_state = state_builder.build(initial_state.get_instant(), coordinates)


Define the Dynamics to consider for Propagation

In [41]:
dynamics = Dynamics.from_environment(environment)


Define a `NumericalSolver` 

In [42]:
numerical_solver = NumericalSolver(
    NumericalSolver.LogType.NoLog,
    NumericalSolver.StepperType.RungeKuttaCashKarp54,
    30.0,
    1.0e-12,
    1.0e-12,
)


Define a `Propagator`

In [43]:
propagator = Propagator(numerical_solver, dynamics)


### Propagate with OSTk

Define the propagation `instants` of interest

In [44]:
instants = Interval.closed(
    start_state.get_instant(), start_state.get_instant() + Duration.seconds(86400.0)
).generate_grid(Duration.seconds(120.0))


Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [45]:
ostk_states = propagator.calculate_states_at(start_state, instants)


---

## Process Cross Platform Results 

Read in reference data from CSV file for GMAT and Orekit

In [46]:
all_comparison_states = []

for ind, filename in enumerate(filenames):
    with open(f"{os.getcwd()}/data/{filename}") as csvfile:
        reader = csv.DictReader(csvfile)
        comparison_states = []
        comparison_instants = []
        for row in reader:
            instant_iter = start_instant + Duration.seconds(
                float(row[reader.fieldnames[0]])
            )
            position_iter = Position.meters(
                [
                    row[reader.fieldnames[1]],
                    row[reader.fieldnames[2]],
                    row[reader.fieldnames[3]],
                ],
                Frame.GCRF(),
            )
            velocity_iter = Velocity.meters_per_second(
                [
                    row[reader.fieldnames[4]],
                    row[reader.fieldnames[5]],
                    row[reader.fieldnames[6]],
                ],
                Frame.GCRF(),
            )

            position_iter_m = np.multiply(
                position_iter.get_coordinates(), multiplication_factors[ind]
            )
            velocity_iter_ms = np.multiply(
                velocity_iter.get_coordinates(), multiplication_factors[ind]
            )

            comparison_instants.append(instant_iter)
            comparison_states.append(
                State(
                    instant_iter,
                    Position.meters(position_iter_m, Frame.GCRF()),
                    Velocity.meters_per_second(velocity_iter_ms, Frame.GCRF()),
                )
            )
    all_comparison_states.append(comparison_states)


#### Trajectory RMS error vs GMAT run in the GCRF frame 

In [47]:
def to_dataframe_RMS(state_ind, list_ind):
    return [
        repr(all_comparison_states[list_ind][state_ind].get_instant()),
        float(
            (
                all_comparison_states[list_ind][state_ind].get_instant()
                - all_comparison_states[list_ind][0].get_instant()
            ).in_seconds()
        ),
        (
            np.linalg.norm(
                (
                    all_comparison_states[list_ind][state_ind]
                    .get_position()
                    .get_coordinates()
                    - ostk_states[state_ind].get_position().get_coordinates()
                )
            )
        ),
        (
            np.linalg.norm(
                (
                    all_comparison_states[list_ind][state_ind]
                    .get_velocity()
                    .get_coordinates()
                    - ostk_states[state_ind].get_velocity().get_coordinates()
                )
            )
        ),
    ]


In [48]:
ostk_states_compared = [
    [
        to_dataframe_RMS(state_ind, list_ind)
        for state_ind in range(0, len(all_comparison_states[list_ind]))
    ]
    for list_ind in range(0, len(all_comparison_states))
]


In [49]:
ostk_states_compared_df = [
    pd.DataFrame(
        ostk_states_compared[list_ind],
        columns=["$Time^{UTC}$", "Elapsed secs", "${\delta}x$", "${\delta}v$"],
    )
    for list_ind in range(0, len(all_comparison_states))
]


# Validation Plots 

Plot position error 

In [50]:
orbit_df_RMS_position_list = [
    ostk_states_compared_df[list_ind][["Elapsed secs", "${\delta}x$"]]
    for list_ind in range(0, len(all_comparison_states))
]

figure = go.Figure()
figure.update_layout(title=f"Position Difference RMS", showlegend=True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Position Difference (m)")

for list_ind, orbit_df_RMS_position in enumerate(orbit_df_RMS_position_list):
    figure.add_trace(
        go.Scatter(
            x=orbit_df_RMS_position["Elapsed secs"],
            y=orbit_df_RMS_position["${\delta}x$"],
            name=legend_name_list[list_ind],
            mode="lines",
        )
    )

figure.show()


Plot velocity error

In [51]:
orbit_df_RMS_velocity_list = [
    ostk_states_compared_df[list_ind][["Elapsed secs", "${\delta}v$"]]
    for list_ind in range(0, len(all_comparison_states))
]

figure = go.Figure()
figure.update_layout(title=f"Velocity Difference RMS", showlegend=True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Velocity Difference (m/s)")

for list_ind, orbit_df_RMS_velocity in enumerate(orbit_df_RMS_velocity_list):
    figure.add_trace(
        go.Scatter(
            x=orbit_df_RMS_velocity["Elapsed secs"],
            y=orbit_df_RMS_velocity["${\delta}v$"],
            name=legend_name_list[list_ind],
            mode="lines",
        )
    )

figure.show()
